In [1]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [2]:
print(100*"#")
%pip show polars
print(100*"-")
%pip show pandas
print(100*"#")
%pip show numpy
print(100*"#")
%pip show matplotlib
print(100*"*")
%pip show huggingface_hub
print(100*"#")
%pip show datasets
print(100*"#")
%pip show pyarrow
print(100*"#")
%pip show pillow
print(100*"#")
%pip show opencv-python
print(100*"#")

####################################################################################################
Name: polars
Version: 1.21.0
Summary: Blazingly fast DataFrame library
Home-page: https://www.pola.rs/
Author: 
Author-email: Ritchie Vink <ritchie46@gmail.com>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: cudf-polars-cu12
Note: you may need to restart the kernel to use updated packages.
----------------------------------------------------------------------------------------------------
Name: pandas
Version: 2.2.3
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: 
Author-email: The Pandas Development Team <pandas-dev@python.org>
License: BSD 3-Clause License

Copyright (c) 2008-2011, AQR Capital Management, LLC, Lambda Foundry, Inc. and PyData Development Team
All rights reserved.

Copyright (c) 2011-2023, Open source contributors.

Redistribution and use in source and bina

In [3]:
import zipfile
import os, sys
import shutil
import json
import random
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import polars as pl
import pyarrow as pa
import cv2
import pickle

import pyarrow.parquet as pq

from PIL import Image

from kaggle_secrets import UserSecretsClient

from datasets import Dataset, DatasetDict

from huggingface_hub import (
    Repository, 
    get_full_repo_name,
    login,
    upload_folder,
    hf_hub_download,
    HfApi
)

In [4]:
def save_pickle(input_save, file_path):
    with open(file_path, "wb") as outfile:
     	# "wb" argument opens the file in binary mode
    	pickle.dump(input_save, outfile)

In [5]:
def load_pickle(file_path):
    with open(file_path, "rb") as infile:
     	record_obj = pickle.load(infile)
    return record_obj

In [6]:
def load_text_file(file_path):
    with open(file_path, "r") as file:
        content = file.read()
    return content

In [17]:
 class_name = [
     "antelope", 
     "grizzly+bear", 
     "killer+whale", 
     "beaver", 
     "dalmatian", 
     "persian+cat", 
     "horse", 
     "german+shepherd", 
     "blue+whale", 
     "siamese+cat", 
     "skunk", 
     "mole", 
     "tiger", 
     "hippopotamus",
     "leopard", 
     "moose", 
     "spider+monkey", 
     "humpback+whale", 
     "elephant", 
     "gorilla", 
     "ox", 
     "fox", 
     "sheep", 
     "seal", 
     "chimpanzee", 
     "hamster", 
     "squirrel", 
     "rhinoceros", 
     "rabbit", 
     "bat", 
     "giraffe", 
     "wolf", 
     "chihuahua", 
     "rat", 
     "weasel", 
     "otter", 
     "buffalo", 
     "zebra", 
     "giant+panda", 
     "deer", 
     "bobcat", 
     "pig", 
     "lion", 
     "mouse", 
     "polar+bear", 
     "collie", 
     "walrus", 
     "raccoon", 
     "cow", 
     "dolphin"
 ]

In [9]:
data01 = load_pickle('/kaggle/input/awa2-dataset/AwA2/pose/Annotations/antelope/antelope_10002.pickle')
data02 = load_pickle('/kaggle/input/awa2-dataset/AwA2/pose/Annotations/buffalo/buffalo_10001.pickle')

In [10]:
data01

{'a1': {'right_antler_base': [501.27272727272725, 258.7272727272727],
  'right_antler_end': [546.7272727272726, 131.9090909090909],
  'left_antler_base': [524.4545454545454, 259.1818181818182],
  'left_antler_end': [575.8181818181818, 137.8181818181818],
  'nose': [486.27272727272725, 331.9090909090909],
  'upper_jaw': [484.9090909090909, 337.3636363636363],
  'lower_jaw': [484.9090909090909, 343.7272727272727],
  'mouth_end_left': [505.3636363636363, 337.8181818181818],
  'left_earbase': [553.090909090909, 271.45454545454544],
  'left_earend': [576.7272727272726, 231.90909090909088],
  'left_eye': [534.0, 292.3636363636363],
  'right_earbase': [493.09090909090907, 258.7272727272727],
  'right_earend': [438.99999999999994, 209.63636363636363],
  'neck_base': [564.0, 312.8181818181818],
  'neck_end': [582.6363636363636, 365.99999999999994],
  'back_base': [590.8181818181818, 364.18181818181813],
  'back_middle': [670.8181818181818, 360.99999999999994],
  'back_end': [778.9999999999999, 

In [11]:
data02

{'a1': {'back_right_thai': [218.93939393939394, 429.7979797979798],
  'back_right_knee': [281.3131313131313, 399.74747474747477],
  'back_right_paw': [350.25252525252523, 487.1212121212121],
  'tail_base': [159.34343434343435, 444.6969696969697],
  'tail_end': [222.72727272727272, 495.95959595959596],
  'front_right_knee': [432.82828282828285, 468.93939393939394],
  'front_right_thai': [389.14141414141415, 432.57575757575756],
  'front_right_paw': [386.11111111111114, 487.8787878787879],
  'throat_base': [461.11111111111114, 440.65656565656565],
  'throat_end': [452.020202020202, 451.01010101010104],
  'nose': [495.45454545454544, 406.06060606060606],
  'upper_jaw': [494.6969696969697, 414.64646464646466],
  'lower_jaw': [494.44444444444446, 419.949494949495],
  'mouth_end_right': [482.57575757575756, 417.92929292929296],
  'mouth_end_left': [501.5151515151515, 419.6969696969697],
  'left_eye': [502.77777777777777, 365.65656565656565],
  'left_antler_base': [505.3030303030303, 345.7070